#Decision Trees


# Definition and Structure

1. Definition: A Decision Tree is a supervised learning algorithm that partitions the feature space into a tree-like structure based on feature values, with the aim of predicting a target variable.
2. Structure: The tree consists of nodes where each node represents a feature test, and branches represent the outcomes of the test leading to subsequent nodes or leaf nodes (terminal nodes) representing the final class prediction.

# Splitting Criteria

* Information Gain: Measures the reduction in entropy or impurity after a dataset is split.
* Gini Impurity: Measures the probability of incorrectly classifying a randomly chosen element if it were randomly classified.

#Training and Prediction

* Training: Recursive partitioning of the data based on feature values to maximize information gain or minimize impurity.
* Prediction: Traversing the tree from the root to a leaf node based on feature values to predict the target class.

#Advantages and Disadvantages

* Advantages: Interpretable, can handle both numerical and categorical data, and relatively efficient for training and inference.
* Disadvantages: Prone to overfitting, sensitive to small variations in the training data.

#Applications

* Classification: Used extensively for classification tasks.
* Regression: Decision Trees can also be used for regression tasks.

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Load the MNIST dataset
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist['data'], mnist['target']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Decision Tree Classifier
dt_classifier = DecisionTreeClassifier()

# Train the model
dt_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = dt_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Decision Tree Accuracy: {accuracy}")

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Decision Tree Accuracy: 0.8711428571428571


#PAC Learning

Probably Approximately Correct (PAC) learning is a theoretical framework in machine learning that provides guarantees on the generalization performance of learning algorithms.

a. Sample Complexity: Understanding how many training examples are needed to ensure a learner's generalization error is within a certain bound.

b. Growth Function: Analysis of the growth function representing the number of distinct labeled datasets that can be realized by a hypothesis class.

c. VC Dimension: The Vapnik-Chervonenkis (VC) dimension measures the capacity of a hypothesis class to shatter points in a dataset, providing insights into the learnability of the class.

d. Generalization Bounds: Theoretical guarantees on the expected difference between training error and true error based on sample size and hypothesis class complexity.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import fetch_openml
from math import log, ceil

def vc_dimension_decision_tree(max_depth, num_features):
    # VC Dimension estimation for Decision Trees
    if max_depth is None:
        # Infinite VC Dimension for unrestricted depth
        return float('inf')
    else:
        # Calculate VC Dimension based on the maximum depth and number of features
        return 2 * max_depth * num_features + 1

def sample_complexity_decision_tree(max_depth, num_features, target_error, confidence):
    # Estimate VC Dimension of the Decision Tree hypothesis class
    vc_dimension = vc_dimension_decision_tree(max_depth, num_features)

    # Calculate sample complexity using VC Dimension
    sample_size = ceil((vc_dimension * log(1 / confidence)) / target_error**2)

    return sample_size

# Load the MNIST dataset
X, y = mnist['data'], mnist['target']

# Determine the number of features in the dataset
num_features = X.shape[1]

# Initialize a DecisionTree Classifier (you can adjust max_depth as needed)
max_depth = 10  # Example: Set the maximum depth of the DecisionTree
dt_classifier = DecisionTreeClassifier(max_depth=max_depth)

# Estimate sample complexity for the DecisionTree Classifier
target_error = 0.05  # Example: Desired error rate (5%)
confidence = 0.95  # Example: Desired confidence level (95%)
sample_size = sample_complexity_decision_tree(max_depth, num_features, target_error, confidence)

print(f"Estimated sample size needed: {sample_size}")

Estimated sample size needed: 321733


#Hyperparameter Tuning for Decision Trees (With Cross-Validation)

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize Decision Tree Classifier
dt_classifier = DecisionTreeClassifier()

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid, cv=3)
grid_search.fit(X_train, y_train)

# Get best parameters and retrain the model
best_params = grid_search.best_params_
best_dt_classifier = DecisionTreeClassifier(**best_params)
best_dt_classifier.fit(X_train, y_train)

# Evaluate on test set
y_pred_tuned = best_dt_classifier.predict(X_test)
accuracy_tuned = accuracy_score(y_test, y_pred_tuned)
print(f"Tuned Decision Tree Accuracy: {accuracy_tuned}")

In [ ]:
# Import necessary libraries
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Split data into features and labels
X, y = mnist.data, mnist.target.astype(int)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Engineering Pipeline
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, slice(0, X.shape[1]))
    ])

# Hyperparameter Optimization
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__max_features': ['auto', 'sqrt', 'log2']
}

# Handling Imbalanced Data - Not implemented in this example

# Ensemble Learning with Random Forest
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', RandomForestClassifier())])

# Grid Search
grid_search = GridSearchCV(model, param_grid, cv=3, verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get best parameters
best_params = grid_search.best_params_

# Train the final model with best parameters
best_model = Pipeline(steps=[('preprocessor', preprocessor),
                             ('classifier', RandomForestClassifier(**best_params))])

best_model.fit(X_train, y_train)

# Make predictions
y_pred = best_model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

#Ensemble Methods (Random Forest)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred_rf = rf_classifier.predict(X_test)

# Calculate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf}")


Random Forest Accuracy: 0.9672857142857143


#Advanced topics on PAC:
1. Online and Active Learning:

Explore online learning algorithms that update the model continuously as new data arrives. Active learning strategies focus on selecting the most informative instances for labeling, reducing the labeling cost while maintaining model performance.

2. Sample Complexity Bounds:

Dive deeper into theoretical analysis of sample complexity bounds for various learning problems. Understand how the sample complexity depends on factors such as the complexity of the hypothesis class, the noise level in the data, and the desired level of confidence.

3. Non-IID Data and Federated Learning:

Study techniques for learning from non-IID (non-identically distributed) data distributions, common in scenarios like federated learning where data is distributed across multiple devices or locations. Federated learning enables training models on decentralized data while preserving privacy and data security.

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
from sklearn.model_selection import train_test_split
from collections import defaultdict

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 28 * 28).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28 * 28).astype('float32') / 255.0

# Define TensorFlow model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Online learning
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1)

# Active learning
def uncertainty_sampling(X_pool, model, n_instances=100):
    probas = model.predict(X_pool)
    entropy = -np.sum(probas * np.log(probas + 1e-8), axis=1)
    idx = np.argsort(entropy)[-n_instances:]
    return idx

X_pool = X_train.copy()
y_pool = y_train.copy()
labeled_idx = []

for _ in range(5):  # Perform 5 active learning iterations
    query_idx = uncertainty_sampling(X_pool, model)
    labeled_idx.extend(query_idx)
    X_labeled = X_pool[query_idx]
    y_labeled = y_pool[query_idx]
    X_pool = np.delete(X_pool, query_idx, axis=0)
    y_pool = np.delete(y_pool, query_idx)
    model.fit(X_labeled, y_labeled, epochs=1, batch_size=32, verbose=0)

# Federated learning
def create_federated_data(X_train, y_train, num_clients=10):
    federated_data = defaultdict(list)
    client_ids = np.random.choice(np.arange(num_clients), size=len(X_train))
    for client_id in range(num_clients):
        client_indices = np.where(client_ids == client_id)[0]
        federated_data[client_id] = (X_train[client_indices], y_train[client_indices])
    return federated_data

federated_data = create_federated_data(X_train, y_train)

def federated_averaging(model, federated_data, num_rounds=10, batch_size=32):
    for _ in range(num_rounds):
        for client_id, (X_client, y_client) in federated_data.items():
            model.fit(X_client, y_client, epochs=1, batch_size=batch_size, verbose=0)
        global_weights = model.get_weights()
        for layer in range(len(global_weights)):
            for client_id, (X_client, _) in federated_data.items():
                client_weights = model.get_weights()
                global_weights[layer] += client_weights[layer] / len(federated_data)
        model.set_weights(global_weights)

federated_averaging(model, federated_data)

# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/5
1688/1688 [==============================] - 18s 9ms/step - loss: 0.2707 - accuracy: 0.9237 - val_loss: 0.1214 - val_accuracy: 0.9647
Epoch 2/5
1688/1688 [==============================] - 12s 7ms/step - loss: 0.1223 - accuracy: 0.9640 - val_loss: 0.1013 - val_accuracy: 0.9705
Epoch 3/5
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0828 - accuracy: 0.9754 - val_loss: 0.0998 - val_accuracy: 0.9695
Epoch 4/5
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0613 - accuracy: 0.9809 - val_loss: 0.0729 - val_accuracy: 0.9787
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 39105.8672 - accuracy: 0.9787
Test loss: 39105.8671875, Test accuracy: 0.9786999821662903
